## Importing the Library

In [ ]:
from fastai.vision.all import *

set the random, torch, and numpy seeds with the set_seed function

In [ ]:
set_seed(16)

## Setting up our data


In [ ]:
path = Path("../input")

#### ls fxn is used to see all files and directories inhere

In [ ]:
path.ls()

In [ ]:
data_path = path/'cassava-leaf-disease-classification'
data_path.ls()

#### loading train.csv with the help of pandas

In [ ]:
df = pd.read_csv(data_path/'train.csv')

#### Sample data

In [ ]:
df.head()

We have an `image_id` and a `label`. We're going to modify our values in `image_id` to make our lives easier when it comes to running inference. 

Why? 


In fastai we have a `get_x` and a `get_y` and this will dictate how it will *always* look for our data, regardless of how it is stored. If we built a `get_y` based on the current `DataFrame`, it would look something like so:

In [ ]:
def get_x(row): return data_path/'train_images'/row['image_id']

In [ ]:
PILImage.create(get_x(df.iloc[0]))

In [ ]:
df.iloc[0]

 ***But*** there is a very large issue here. We always have our `get_x` tied to the training directory which makes it more complicated for us to work with our `test_images` directory.

What's the solution? 

Add `train_images` into the dataframe through a `lambda` function:

In [ ]:
df.head()

In [ ]:
df['image_id'] = df['image_id'].apply(lambda x: f'train_images/{x}')

In [ ]:
df.head()

Now we won't run into an issue when we're testing. 

### Adjusting our label

What else can we do?

Let's change our lables into something more readable through a dictionary (these come from the `json` file):

In [ ]:
idx2lbl = {0:"Cassava Bacterial Blight (CBB)",
          1:"Cassava Brown Streak Disease (CBSD)",
          2:"Cassava Green Mottle (CGM)",
          3:"Cassava Mosaic Disease (CMD)",
          4:"Healthy"}

df['label'].replace(idx2lbl, inplace=True)

In [ ]:
df.head()

## Building the `DataBlock`
​
Let's think about how our problem looks. `fastai` provides blocks to center around *most* situations, and this is no exception.
​
We know our input is an image and our output is a category, so let's use `ImageBlock` and `CategoryBlock`:

In [ ]:
blocks = (ImageBlock, CategoryBlock)

Next we'll want to split our data somehow. We'll use a `RandomSplitter` and split our data 80/20

In [ ]:
splitter = RandomSplitter(valid_pct=0.2)

Our `DataBlock` is also going to want to know how to get our data. Since our data all stems from a `csv`, we will make a `get_x` and `get_y` function:
(we already made our `get_x`)

In [ ]:
def get_x(row): return data_path/row['image_id']

def get_y(row): return row['label']

We can see that when we write custom `get_` functions, it will accept one *row* of our `DataFrame` to look at, and so we can filter as a result.

Next we'll come up with some basic data augmentations. 

Our `item_tfms` should ensure everything is ready to go into a batch, so we will use `Resize`.

Our `batch_tfms` should apply any extra augmentations we may want. We'll use `RandomResizedCropGPU`, `aug_transforms`, and apply our `Normalize`:
> We will normalize our data based on ImageNet, since that is what our pretrained model was trained with

In [ ]:
item_tfms = [Resize(448)]
batch_tfms = [RandomResizedCropGPU(224), *aug_transforms(), Normalize.from_stats(*imagenet_stats)]

Lets Build our first DataBlock

In [ ]:
block = DataBlock(blocks = blocks,
                 get_x = get_x,
                 get_y = get_y,
                 splitter = splitter,
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms)

And now we can turn this into some `DataLoaders`. We're going to pass in some items (which in our case is our `DataFrame`) and a batch size to use. We will use 64:

In [ ]:
dls = block.dataloaders(df, bs=64)

In [ ]:
dls.show_batch(figsize=(12,12))

## Training Model

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'

Now that our weights are setup, let's look at how to use `cnn_learner`. We're going to use a few tricks during our training that fastai can help us out with. 

Specifically we will be using the `ranger` optimizer function and `LabelSmoothingCrossEntropy` as our loss function.

Along with these we'll be using the `accuracy` metric as this is how this competition will grade our results with:

In [ ]:
learn = cnn_learner(dls, resnet50, opt_func=ranger, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

In [ ]:
def fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tune with `freeze` for `freeze_epochs` then with `unfreeze` from `epochs` using discriminative LR"
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)

In [ ]:
@patch
def fine_tune_flat(self:Learner, epochs, base_lr=4e-3, freeze_epochs=1, lr_mult=100, pct_start=0.75, 
                   first_callbacks = [], second_callbacks = [], **kwargs):
    "Fine-tune applied to `fit_flat_cos`"
    self.freeze()
    self.fit_flat_cos(freeze_epochs, slice(base_lr), pct_start=0.99, cbs=first_callbacks, **kwargs)
    base_lr /= 2
    self.unfreeze()
    self.fit_flat_cos(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, cbs=second_callbacks)

In [ ]:
learn.lr_find()

In [ ]:
cbs1 = [MixUp(alpha = 0.7)]
cbs2 = [MixUp(alpha = 0.3)]

In [ ]:
learn.fine_tune_flat(5, base_lr=1e-3, pct_start=0.72, first_callbacks=cbs1, second_callbacks=cbs2)

## Submitting some results


Let's look at the sample submission dataframe first:

In [ ]:
sample_df = pd.read_csv(data_path/'sample_submission.csv')
sample_df.head()

Let's look at the sample submission dataframe first:

In [ ]:
sample_copy = sample_df.copy()

In [ ]:
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'test_images/{x}')

Next we'll make an inference dataloader through the `test_dl` method:

In [ ]:
test_dl = learn.dls.test_dl(sample_copy)

We'll look at a batch of data to make sure it all looks okay:

In [ ]:
test_dl.show_batch()

Next we'll grab some predictions. We will use the `.tta` method to run test-time-augmentation which can help boost our accuracy some:

In [ ]:
preds, _ = learn.tta(dl=test_dl)

And now we can submit them:

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample_df.to_csv('submission.csv',index=False)